In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kraft

In [ ]:
na_pa = {}

na_se_li_ = {}

for pa in kraft.path.get_child("../raw/"):

    if pa[-3:] != ".md":

        continue

    na = pa.split("/")[-1][: -len(".md")]

    assert na == kraft.string.untitle(na)

    na_pa[na] = pa

    se_ = [".", "<", ">"]

    se_li_ = {se: [] for se in se_}

    with open(pa) as io:

        for li in io:

            li = li.strip()

            if li == "":

                continue

            if li[0] == "#":

                se = li[2]

                assert se_.pop(0) == se

                continue

            # TODO: remove ()
            if li[0] == "(":

                assert li[-2] == "."

            se_li_[se].append(li)

    na_se_li_[na] = se_li_

len(na_se_li_)

In [ ]:
for na, se_li_ in na_se_li_.items():

    for li in se_li_["<"]:

        assert kraft.string.untitle(li) not in na_se_li_, "{}: {}".format(na, li)

In [ ]:
import pandas as pd

In [ ]:
fe_pr = pd.read_csv(
    "../feature.tsv", "\t", index_col=0, squeeze=True, dtype=str
).to_dict()

In [ ]:
for se_li_ in na_se_li_.values():

    se_li_["md<"] = []

    se_li_[">md"] = []

for na, se_li_ in na_se_li_.items():

    for li in list(se_li_[">"]):

        limi = kraft.string.untitle(li)

        if limi in na_se_li_:

            assert li not in fe_pr, li

            se_li_[">"].remove(li)

            se_li_[">md"].append(li)

            na_se_li_[limi]["md<"].append(na)

In [ ]:
def combine_right(se_li_):

    return se_li_[">"] + kraft.iterable.flatten(
        [na_se_li_[kraft.string.untitle(li)][">"] for li in se_li_[">md"]]
    )

In [ ]:
for na, se_li_ in na_se_li_.items():

    for li, n in pd.value_counts(combine_right(se_li_)).items():

        if 1 < n:

            so_ = [na]

            so_ += [
                liri
                for liri in se_li_[">md"]
                if li in na_se_li_[kraft.string.untitle(liri)][">"]
            ]

            print("{}:\n\t{}\n".format(li, "\n\t".join(so_)))

In [ ]:
fe_ = []

for se_li_ in na_se_li_.values():

    fe_ += se_li_["<"] + se_li_[">"]

In [ ]:
mi_ = []

for fe in fe_pr.keys():

    if fe not in fe_:

        mi_.append(fe)

mi_

In [ ]:
ne_ = []

for na, se_li_ in na_se_li_.items():

    for fe in se_li_["<"] + se_li_[">"]:

        if fe not in fe_pr:

            ne_.append([na, fe])

assert len(ne_) == 0, ne_

In [ ]:
import os
import shutil

In [ ]:
de = "\n\n"

In [ ]:
def number(pr):

    return [int(it) for it in pr.split(".")]

In [ ]:
def order(se):

    return [number(pr) + [fe] for fe, pr in se.items()]

In [ ]:
na_na2 = {}

In [ ]:
for na, na2 in na_na2.items():

    fe_pr[na2] = fe_pr[na]

In [ ]:
se = pd.Series(fe_pr).sort_values(key=order)

se.index.name = "Feature"

se.name = "Priority"

se.to_csv("../feature.tsv", "\t")

In [ ]:
di = "../raw/"

for na, se_li_ in na_se_li_.items():

    pa = na_pa[na].replace("../raw/", di)

    kraft.path.make(pa, pr=False)

    with open(pa, "w") as io:

        ti_li_ = {
            ".": [],
            "<": [],
            ">": [],
        }

        for li in se_li_["."]:

            ti_li_["."].append(li)

        for li in se_li_["<"]:

            ti_li_["<"].append(na_na2.get(li, li))

        for li in se_li_[">"] + se_li_[">md"]:

            ti_li_[">"].append(na_na2.get(li, li))

        for ti, li_ in ti_li_.items():

            io.write("# {}".format(ti) + de)

            if len(li_) == 0:

                continue

            io.write(de.join(li_) + de)

In [ ]:
def prioritize(fe):

    if fe.endswith(".") or kraft.string.untitle(fe) in na_se_li_:

        return number("0.0.0") + [fe]

    return number(fe_pr[fe]) + [fe]

In [ ]:
di = "../compiled/"

if os.path.isdir(di):

    shutil.rmtree(di)

for na, se_li_ in na_se_li_.items():

    pa = na_pa[na].replace("../raw/", di)

    kraft.path.make(pa, pr=False)

    with open(pa, "w") as io:

        ti_li_ = {
            "Comment": [],
            "Epidemiology": [],
            "Cause": [],
            "Symptom": [],
            "Sign": [],
            "Diagnostic": [],
            # "Treatment and Management": [],
        }

        for li in se_li_["."]:

            ti_li_["Comment"].append(li)

        for li in se_li_["<"]:

            if 2 <= number(fe_pr[li])[0] <= 5:

                ti_li_["Epidemiology"].append(li)

        for li in se_li_["md<"]:

            ti_li_["Cause"].append(kraft.string.title(li))

        for li in combine_right(se_li_):

            if number(fe_pr[li])[0] == 1:

                ti_li_["Symptom"].append(li)

            elif number(fe_pr[li])[0] == 6:

                ti_li_["Sign"].append(li)

            elif 7 <= number(fe_pr[li])[0]:

                ti_li_["Diagnostic"].append(li)

        for ti, li_ in ti_li_.items():

            io.write("# {}".format(ti) + de)

            if len(li_) == 0:

                continue

            io.write(de.join(sorted(li_, key=prioritize)) + de)